In [1]:
%reload_ext autoreload
%autoreload 2

from interactive import utils, config, visualisation

Config = config.Config()

# Welcome to TESSERA!

In this interactive notebook you can visualise the embeddings for a region of interest (ROI) and train a [k-Nearest Neigbours](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) classifier.

We hope this will demonstrate the power of TESSERA's openly-available embeddings for performing land cover classification over a range of areas and cover classes.

### How to use

1. Run the previous cell (by clicking the 'play' button) to load the necessary code
2. Update your region of interest (ROI) in the cell below
3. Run the cell below!
4. When the map appears:
   - Follow the interface to select points on the map for each class
   - Add new classes as necessary
   - Click 'classify' to see the result of the k-Nearest Neigbours classifier
   - Export your points by clicking 'Save labels'

This product is in early stages: please get in touch with any feedback.

In [2]:
# -- 1. ROI DEFINITION --

# This is in increments of 0.1, and requires at least a 0.1 by 0.1 deg bbox
# **USER INPUT REQUIRED** update coordinates here, otherwise defaults will be used (South Cambridge)
MIN_LON, MAX_LON = -0.10, 0.30
MIN_LAT, MAX_LAT = 52.00, 52.20

# -- 2. FETCH AND MOSAIC RELEVANT TESSERA TILES --

embedding_mosaic, mosaic_transform = utils.TesseraUtils().process_roi_to_mosaic(
    lat_coords=(MIN_LAT, MAX_LAT),
    lon_coords=(MIN_LON, MAX_LON),
)

# -- 3. VISUALISE MAP, PLACE TRAINING POINTS, & RUN CLASSIFICATION --

mapping_tool = visualisation.InteractiveMappingTool(
    MIN_LAT,
    MAX_LAT,
    MIN_LON,
    MAX_LON,
    embedding_mosaic,
    mosaic_transform,
)
mapping_tool.display()

Bounding box defined:
┗ (52.00, -0.10) | ┓ (52.20, 0.30)

Searching for tiles in ROI: (-0.1, 52.0, 0.3, 52.2) for year 2024

Found 15 tiles to merge.


Processing tiles:   0%|          | 0/15 [00:00<?, ?it/s]


Merging all tiles...
Shape of final embedding mosaic: (2853, 4791, 128)
Bounds of displayed embedding mosaic: ┗ (51.90, -0.21) | ┓ (52.20, 0.31)

Creating PCA-based visualization...
Normalizing PCA components for display...
PCA visualization created.
Image overlay added to map.
